---

<img src="images/anchormen-logo.png" width="500">

---

## Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import mplleaflet

%matplotlib inline

## Functions

In [2]:
def unixtime_to_datetime(time):
    if(np.isnan(time)):
        return np.nan
    else:
        return(datetime.datetime.fromtimestamp(time))

def clean_opensky_dataframe(df):
    # Create variables from 'states' list
    df['icao24'] = df.apply(lambda row: row.states[0], axis=1)
    df['callsign'] = df.apply(lambda row: row.states[1], axis=1)
    df['origin_country'] = df.apply(lambda row: row.states[2], axis=1)
    df['time_position'] = df.apply(lambda row: row.states[3], axis=1)
    df['time_velocity'] = df.apply(lambda row: row.states[4], axis=1)
    df['longitude'] = df.apply(lambda row: row.states[5], axis=1)
    df['latitude'] = df.apply(lambda row: row.states[6], axis=1)
    df['altitude'] = df.apply(lambda row: row.states[7], axis=1)
    df['on_ground'] = df.apply(lambda row: row.states[8], axis=1)
    df['velocity'] = df.apply(lambda row: row.states[9], axis=1)
    df['heading'] = df.apply(lambda row: row.states[10], axis=1)
    df['vertical_rate'] = df.apply(lambda row: row.states[11], axis=1)
    df['sensors'] = df.apply(lambda row: row.states[12], axis=1)

    # Drop 'states'
    df.drop('states', axis=1, inplace=True)

    # Convert timestamps
    df['time'] = df['time'].apply(unixtime_to_datetime)
    df['time_position'] = df['time_position'].apply(unixtime_to_datetime)
    df['time_velocity'] = df['time_velocity'].apply(unixtime_to_datetime)
    
    # Remove entries with unknown longitude or latitude
    df = df.dropna(subset=['longitude', 'latitude'])
    
    return(df)

## Retrieve Data from REST API

See documentation: https://opensky-network.org/apidoc/rest.html

In [3]:
# Read from OpenSky API
df_airplanes = pd.read_json("https://opensky-network.org/api/states/all")

# Clean dataframe
df_airplanes = clean_opensky_dataframe(df_airplanes)
df_airplanes.head(3)

,time,icao24,callsign,origin_country,time_position,time_velocity,longitude,latitude,altitude,on_ground,velocity,heading,vertical_rate,sensors
0,2021-05-13 11:49:00,511142,ESTWN,Estonia,2021-05-13 11:48:59,2021-05-13 11:48:59,23.4247,60.3358,3048.0,False,102.54,276.63,0.00,None
1,2021-05-13 11:49:00,e8027b,LAN9581,Chile,2021-05-13 11:47:43,2021-05-13 11:47:43,-79.7785,-2.8193,12496.8,False,252.43,172.98,0.00,None
2,2021-05-13 11:49:00,4b1812,SWR2060,Switzerland,2021-05-13 11:48:59,2021-05-13 11:48:59,2.1106,44.5967,11277.6,False,214.48,237.34,0.65,None


## Visualization: plot airplanes on an interactive leaflet background

In [4]:
# Interactive visualization
fig, ax = plt.subplots(figsize=(16,16))
df_airplanes.plot(x='longitude', y='latitude', kind='scatter', color='red', s=2, ax=ax)
mplleaflet.display(fig=fig)

AttributeError: 'XAxis' object has no attribute '_gridOnMajor'

### Folium Maps

In [5]:
# Visualization functions
import folium 
from folium import plugins

def visualize_points_folium(df, x_coord='longitude', y_coord='latitude', name_coord='callsign', zoom = 4):
    """Visualize lat/lon locations on map using Folium"""
    
    # Bit of cleaning
    df = df.dropna(subset=[x_coord, y_coord])
    
    # Determine mean lon/lat coordinates
    center_lon, center_lat = df[[x_coord, y_coord]].mean()

    # Initialize map
    map = folium.Map(location = [center_lat, center_lon], zoom_start = zoom, tiles='Stamen Terrain')
 
    # Instantiate a mark cluster object for the locations in the dataframe
    locations = plugins.MarkerCluster().add_to(map)
 
    # Loop through the dataframe and add each data point to the mark cluster
    for lon, lat, name, in zip(df[x_coord], df[y_coord], df[name_coord]):
        
        folium.Marker(
            location=[lat, lon],
            icon=None,
            popup = folium.Popup(name),
        ).add_to(locations)
    
    # Display map
    return(map)

In [6]:
# Read from OpenSky API
df_airplanes = pd.read_json("https://opensky-network.org/api/states/all")

# Clean dataframe
df_airplanes = clean_opensky_dataframe(df_airplanes).head(1000)

visualize_points_folium(df_airplanes, 
                        x_coord='longitude', 
                        y_coord='latitude', 
                        name_coord='callsign',
                        zoom = 4)